# LCO/LPO

In [93]:
from interfacemaster.cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, get_normal_from_MI, rot, MID
from interfacemaster.interface_generator import core, convert_vector_index, get_disorientation
from numpy import array, dot, round, cross, eye, pi
from numpy.linalg import inv, det, norm

In [86]:
from pymatgen.core.structure import Structure

In [119]:
my_interface = core('cif_files/beta/CONTCAR',\
                    'cif_files/beta/CONTCAR')

Warning!, this programme will rewrite the POSCAR file in this dir!


In [120]:
my_interface.conv_lattice_1

array([[ 3.10413533e-08,  3.09981602e+00, -2.60876841e-08],
       [ 1.25126373e+01,  7.74478610e-09, -1.40914748e+00],
       [-2.50500860e-03,  1.21877777e-08, -5.74305976e+00]])

In [121]:
v = get_normal_from_MI(lattice=my_interface.conv_lattice_1, hkl=[1,0,0])

In [122]:
axis = convert_vector_index(my_interface.conv_lattice_1, my_interface.lattice_1, v)
axis

array([-2.73043932e-08,  1.78024287e+01, -4.36809795e+00])

In [123]:
#rotation matrix
R = rot(v,180/180*pi)

In [124]:
my_interface.parse_limit(du = 0.01, S  = 0.01, sgm1=20, sgm2=20, dd = 0.01)

#Do searching!
#We already know that there is a CSL by rotation 180 degrees
my_interface.search_fixed(R = R,exact=True)

Congrates, we found an appx CSL!

U1 = 
[[ 0  0  1]
 [-1  0  0]
 [ 0 -1  1]]; sigma_1 = 1

U2 = 
[[0 0 1]
 [1 0 0]
 [0 1 0]]; sigma_2 = 1

D = 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]




In [105]:
cell_1 = dot(my_interface.lattice_1, my_interface.bicrystal_U1)

In [106]:
cell_2 = dot(my_interface.lattice_1, my_interface.bicrystal_U2)

In [125]:
hkl = get_primitive_hkl(hkl=[0,0,1],C_lattice=my_interface.conv_lattice_1,P_lattice=my_interface.lattice_1)
hkl

array([0, 0, 1])

In [134]:
z = array([-5.9585793637, 7.20003761e-08, 24.3055493118])

In [135]:
norm(z) + 10

35.02527513099213

In [136]:
35.02527513099213/norm(z) * z

array([-8.33960388e+00,  1.00771439e-07,  3.40179497e+01])

In [126]:
hkl

array([0, 0, 1])

## Get bicrystal

In [127]:
my_interface.compute_bicrystal(hkl=hkl, normal_ortho = False, lim = 50, tol_ortho = 1e-3, tol_integer=1e-3)

hkl in CSL: [ 0 -1  1]
cell 1:
[[ 0  0 -1]
 [ 0 -1  0]
 [-1  0  0]]
cell 2:
[[ 0  0 -1]
 [ 0  1  0]
 [ 1  0 -1]]


In [133]:
my_interface.get_bicrystal(xyz_1 = [3,2,2], xyz_2 = [3,2,2], dp1 = 0, dp2 = 0, dx = 0, vx=0)

In [16]:
norm(my_interface.CNID[:,0])

3.0998160237290167

In [18]:
norm(my_interface.CNID[:,1])

5.715762230300908

In [20]:
#To sample the CNID, try
my_interface.sample_CNID(grid = [3,6], xyz_1 = [x1,1,1], xyz_2 = [x2,1,1], dp1 = 0.1, dp2 = 0, dx = -0.5, vx=0.8)

CNID
[[ 0.          1.33333333]
 [-1.          0.66666667]
 [ 0.         -0.33333333]]
making 18 files...
completed


In [133]:
right_1 = array([24.7867897178, 0.9095271859, 13.7594486662])
right_2 = array([37.0678811529, 2.3316922917, 15.2692225735])

In [134]:
middle = ((right_1 + right_2)/2)[1:3]
r_m_y, r_m_z = middle
print(r_m_y, r_m_z)

1.6206097388 14.514335619850002


In [135]:
left_1 = array([22.2413190086999997, 1.4243551294000001, 12.8611351063000008])

In [136]:
left_2 = array([12.1592408800000005, 2.8491354397999999, 11.4306350467000009])

In [137]:
middle = ((left_1 + left_2)/2)[1:3]
l_m_y, l_m_z = middle

In [138]:
print(l_m_y, l_m_z)

2.1367452846 12.1458850765


In [139]:
dsp = array([0, l_m_y - r_m_y, l_m_z - r_m_z])

In [140]:
dsp

array([ 0.        ,  0.51613555, -2.36845054])

In [141]:
with open('lmp.dat','r') as f:
    lines = f.readlines()
    xhi = float(lines[3].split()[1])
    yhi = float(lines[4].split()[1])
    zhi = float(lines[5].split()[1])
    xy = float(lines[6].split()[0])
    xz = float(lines[6].split()[1])
    yz = float(lines[6].split()[2])

In [142]:
atoms = loadtxt('lmp.dat', skiprows=10)

In [143]:
ids = atoms[:,0]
types = atoms[:,1]
atoms = atoms[:,2:5]

In [144]:
atoms[atoms[:,0] > 23] += dsp

In [145]:
with open('lmp_dsp.dat','w') as f:
    f.write('#Head \n')
    f.write('{} atoms \n'.format(len(atoms)))
    f.write('4 atom types \n')
    f.write(f'0 {xhi} xlo xhi \n')
    f.write(f'0 {yhi} ylo yhi \n')
    f.write(f'0 {zhi} zlo zhi \n')
    f.write(f'{xy} {xz} {yz} xy xz yz \n\n')
    f.write('Atoms # atomic\n\n')
    FinalMat = column_stack((ids, types, atoms))
    savetxt(f, FinalMat, fmt='%i %i %.16f %.16f %.16f')
            
            

In [146]:
left_b = 12.0525
cell_min = 12

In [147]:
left_max = 22.3482
right_min = 24.3096
right_max = 37.5451

In [148]:
right_b = right_max + (right_min - left_max) - (left_b - cell_min)

In [149]:
right_b

39.454